## Calculate a Loan Instalment Schedule 3
Given start-date, principle, number of instalment and interest rate (per year):

* Calculates the EMI (equal monthly instalment) amount 
* Calculates the instalment amortisation schedule
* Maps to a DBEI (optimised) loan in Mambu with Days-in-year: actual/365
* Also supports calculating Mambu's DBEI (non-optimised) with Days-in-year: actual/365
    * NOTE: This uses an excel PMT calculation to work out EMI and adds extras on the last instalment
* Also supports accruing interest daily using compound interest method



In [26]:
import sympy as sp
import sys
import datetime
from dateutil import parser
import calendar
# Need to increase the recusion limit (else .subs fails for 300 instalments)
sys.setrecursionlimit(5000)

In [70]:
def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year,month)[1])
    return datetime.date(year, month, day)

def daysInPeriod(startDate):
    if isinstance(startDate,datetime.date):
        dt = startDate
    else:
        dt = parser.parse(startDate).date()
    dt2 = add_months(dt, 1)
    daysInPeriod = 0
    daysInPeriod = dt2 - dt
    return (dt,dt2,daysInPeriod.days)

def monthlyPeriodInterest(days, interestPerYearPercent, daysCalcMethod):
    if daysCalcMethod == "30/360":
        periodInterest = interestPerYearPercent / 1200
    elif daysCalcMethod == "actual/365":
        periodInterest = days * (interestPerYearPercent / 36500)
    elif daysCalcMethod == "30.42/365":
        periodInterest = 30.42 * (interestPerYearPercent / 36500)
    else:
        assert True == False, "Unknown daysCalcMethod: {0}".format(daysCalcMethod)
        
    return periodInterest

def daysInPeriod2(startDate, interestPerYearPercent, daysCalcMethod):
    dateStuff = daysInPeriod(startDate)
    periodInterest = monthlyPeriodInterest(dateStuff[2], interestPerYearPercent, daysCalcMethod)
    # classic python trick here if tuple only has 1 element - add ,
    dateStuff = dateStuff + (periodInterest,)
    return dateStuff
        


In [328]:
daysInPeriod2("01 Feb 2019",12,'actual/365')

(datetime.date(2019, 2, 1),
 datetime.date(2019, 3, 1),
 28,
 0.009205479452054795)

In [69]:
def schedule(startDate, numInstalments, annualInterest, daysCalcMethod, compoundDailyInterest=False):
    DAYS_CALC_METHOD = daysCalcMethod # See monthlyPeriodInterest above for supported daysCalcMethod
    P, r, n, E = sp.symbols('P r n E')
    dateStuff = daysInPeriod2(startDate, annualInterest, DAYS_CALC_METHOD)
    if compoundDailyInterest == True:
        # using A = P(1+r/n)^nt formulae for compound interest
        intExpected = P*((1 + dateStuff[3]/dateStuff[2]) ** dateStuff[2]) - P
    else:
        intExpected = P*dateStuff[3]
    instalmentList = [{'num':1,'from_date':dateStuff[0], 'days_in_period': dateStuff[2],
                       'interest_expected':intExpected,'principle_remaining':P*(1+dateStuff[3])-E}]
    dateStuff = daysInPeriod2(dateStuff[1], annualInterest, DAYS_CALC_METHOD)
    monthlyPeriodInterest(dateStuff[2],annualInterest,'30/360')
    for i in range(1,numInstalments):
        instObj = {}
        instObj["num"] = i+1
        if compoundDailyInterest == True:
            # using A = P(1+r/n)^nt formulae for compound interest
            instObj["interest_expected"] = instalmentList[i-1]['principle_remaining'] * ((1 + dateStuff[3]/dateStuff[2]) ** dateStuff[2]) - instalmentList[i-1]['principle_remaining']  
        else:
            instObj["interest_expected"] = instalmentList[i-1]['principle_remaining'] * dateStuff[3]
        instObj["principle_remaining"] = instalmentList[i-1]['principle_remaining']+instObj["interest_expected"]-E
        instObj["from_date"] = dateStuff[0]
        instObj["days_in_period"] = dateStuff[2]
        instalmentList.append(instObj)
        dateStuff = daysInPeriod2(dateStuff[1], annualInterest, DAYS_CALC_METHOD)
    return instalmentList

In [63]:
def expandSchedule(startDate, OrigPrinciple, annualInterest, numInstalments, daysCalcMethod, compoundDailyInterest=False, daysCalcMethodEMI=None):
    ROUND_NUMDIGITS = 10
    P, r, n, E = sp.symbols('P r n E')
    shList = schedule(startDate, numInstalments, annualInterest, daysCalcMethod, compoundDailyInterest)
    
    # Solve the Equation for E - for the last instalment
    if daysCalcMethodEMI is None:
        prin = shList[numInstalments-1]['principle_remaining']
        expandedPrin = prin.subs(P,OrigPrinciple)
        equalMonthlyInstalment = sp.solve(expandedPrin, E)[0]
    else:
        # This is to simulate Mambu's standard DBEI (i.e. without optimised setting)
        # Mambu calculates the EMI using standard excel PMT style equation - i.e. daysCalcMethodEMI=="30/360"
        shList2 = schedule(startDate, numInstalments, annualInterest, daysCalcMethodEMI, compoundDailyInterest)
        prin = shList2[numInstalments-1]['principle_remaining']
        expandedPrin = prin.subs(P,OrigPrinciple)
        equalMonthlyInstalment = sp.solve(expandedPrin, E)[0]
    
    # Now gothrough the complete shList and plug in values for all the variables
    for i in range(numInstalments):
        instObj = shList[i]
        instObj["interest_expected"] = round(instObj["interest_expected"].subs(
            {
                P:OrigPrinciple,
                E:equalMonthlyInstalment
            }), ROUND_NUMDIGITS)
        instObj["principle_remaining"] = round(instObj["principle_remaining"].subs(
            {
                P:OrigPrinciple,
                E:equalMonthlyInstalment
            }), ROUND_NUMDIGITS)
        
        if i == 0:
            previousPrinciple = OrigPrinciple
        else:
            previousPrinciple = shList[i-1]["principle_remaining"]
        instObj["principle_expected"] = previousPrinciple - instObj["principle_remaining"]
        
        instObj["total_expected"] = instObj["principle_expected"] + instObj["interest_expected"]

    # Make sure we have fully paid off the loan in the last instalment   
    # There may be some extras to pay when using Mambu's standard DBEI calc
    lastInstalmentObj = shList[numInstalments-1]
    lastInstalmentObj['principle_expected'] = lastInstalmentObj['principle_expected'] + lastInstalmentObj['principle_remaining']
    lastInstalmentObj["total_expected"] = lastInstalmentObj["principle_expected"] + lastInstalmentObj["interest_expected"]
    return (equalMonthlyInstalment,shList)

In [30]:
def outputSchedule(shList,outFile="Schedule.csv"):
    outFile = open("Schedule.csv","w")
    outFile.write("num, from_date, days_in_period, interest_expected, principle_remaining, principle_expected, total_expected\n")
    for instalObj in shList:
        outFile.write("{0}, {1}, {2}, {3}, {4}, {5}, {6}\n".format(
            instalObj['num'], instalObj['from_date'], instalObj['days_in_period'], instalObj['interest_expected'], 
            instalObj['principle_remaining'], instalObj['principle_expected'], instalObj['total_expected']
        ))
        
        
         

In [90]:
#expandSchedule("16 Dec 2019", OrigPrinciple=5000, annualInterest=12, numInstalments=5, daysCalcMethod="actual/365", daysCalcMethodEMI="30/360" )
shList = expandSchedule("16 Dec 2019", OrigPrinciple=755000, annualInterest=4, numInstalments=300, daysCalcMethod="30/360", compoundDailyInterest=False)
#expandSchedule("01 Dec 2019", OrigPrinciple=5000, annualInterest=12, numInstalments=5, daysCalcMethod="30/360" )

In [92]:
outputSchedule(shList[1], outFile="Schedule.csv")

In [91]:
shList

(3985.16814424820,
 [{'num': 1,
   'from_date': datetime.date(2019, 12, 16),
   'days_in_period': 31,
   'interest_expected': 2516.6666666667,
   'principle_remaining': 753531.498522419,
   'principle_expected': 1468.50147758098,
   'total_expected': 3985.16814424768},
  {'num': 2,
   'interest_expected': 2511.7716617414,
   'principle_remaining': 752058.102039912,
   'from_date': datetime.date(2020, 1, 16),
   'days_in_period': 31,
   'principle_expected': 1473.39648250700,
   'total_expected': 3985.16814424840},
  {'num': 3,
   'interest_expected': 2506.8603401330,
   'principle_remaining': 750579.794235796,
   'from_date': datetime.date(2020, 2, 16),
   'days_in_period': 29,
   'principle_expected': 1478.30780411605,
   'total_expected': 3985.16814424905},
  {'num': 4,
   'interest_expected': 2501.9326474527,
   'principle_remaining': 749096.558739001,
   'from_date': datetime.date(2020, 3, 16),
   'days_in_period': 31,
   'principle_expected': 1483.23549679492,
   'total_expected':